In [ ]:
# http://host.robots.ox.ac.uk/pascal/VOC/voc2012/index.html

# http://preview.d2l.ai/d2l-en/master/chapter_computer-vision/semantic-segmentation-and-dataset.html#the-pascal-voc2012-semantic-segmentation-dataset

environment

In [ ]:
using Pkg
envpath = expanduser("~/envs/deeptraining/")
Pkg.activate(envpath)

libraries

In [ ]:
# private libraries
using PascalVocTools; pv = PascalVocTools

In [ ]:
# julia libraries
using Images
using FLoops

load dataset: focus on Class Segmentation

In [ ]:
# paths
datasetPath = expanduser("/scratch/cirobr/datasets/pascalvoc/VOCdevkit/VOC2012/")

imagesPath  = datasetPath * "JPEGImages/"
classesPath = datasetPath * "SegmentationClass/"
objectsPath = datasetPath * "SegmentationObject/"

In [ ]:
# segmentation dataset
# trainFilesTxt = datasetPath * "ImageSets/Segmentation/train.txt"
# validFilesTxt = datasetPath * "ImageSets/Segmentation/val.txt"
trvalFilesTxt = datasetPath * "ImageSets/Segmentation/trainval.txt"


trvalImagesFpfn  = ( imagesPath  .* readlines(trvalFilesTxt) .* ".jpg" )
trvalClassesFpfn = ( classesPath .* readlines(trvalFilesTxt) .* ".png" )
trvalObjectsFpfn = ( objectsPath .* readlines(trvalFilesTxt) .* ".png" );

In [ ]:
k = 1
im = Images.load(trvalImagesFpfn[k])
cl = Images.load(trvalClassesFpfn[k])
ob = Images.load(trvalObjectsFpfn[k])

mosaicview(im, cl, ob, nrow=1)

In [ ]:
eltype(cl)

picture size: recommended padding (500,500), or better (512,512)

In [ ]:
N = length(trvalImagesFpfn)
sizes = Array{Int, 2}(undef, (N,2))

@floop for k in 1:N
    im = Images.load(trvalImagesFpfn[k])
    sizes[k,1], sizes[k,2] = size(im)
end

minimum(sizes[:,1]), maximum(sizes[:,1])

In [ ]:
minimum(sizes[:,2]), maximum(sizes[:,2])

PascalVOC functions

In [ ]:
pv.vec_classes   # 10 => "cow"

In [ ]:
pv.vec_classnames   # "cow" => 10

In [ ]:
pv.vec_colormap   # [64, 128, 0]    => 10

In [ ]:
pv.voc_classname2classnumber["cow"], pv.voc_colormap2classnumber[[64,128,0]]

In [ ]:
# Return the value stored for the given key, or the given default value if no mapping for the key is present.
# get(collection, key, default)
get(pv.voc_colormap2classnumber, [224,224,192], 0)   # edges

mask conversion

In [ ]:
typeof(cl)

In [ ]:
cl

In [ ]:
cl_out = pv.voc_rgb2classes(cl)
unique(cl_out)

In [ ]:
[pv.voc_classnumber2classname[k] for k in unique(cl_out)]